Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give read access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [ ]:
## DONT CHANGE THIS CELL 
# this is currently the same as dev.data.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt

--2021-10-25 17:01:43--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt [following]
--2021-10-25 17:01:44--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63639 (62K) [text/plain]
Saving to: ‘test.data.txt’

test.data.txt       100%[===================>]  62.15K  76.3KB/s    in 0.8s    

2021-10-25 17:01:46 (76.3 KB/s) - ‘test.data.txt’ saved [63639/63639]



In [ ]:
## Replace with the right link that contains the zip file uploaded from the training
!gdown https://drive.google.com/uc?id=1-AiCD72t0b6t5gNumxQuZTOu3NXoDwL_

# https://drive.google.com/file/d/1-AiCD72t0b6t5gNumxQuZTOu3NXoDwL_/view?usp=sharing


Downloading...
From: https://drive.google.com/uc?id=1-AiCD72t0b6t5gNumxQuZTOu3NXoDwL_
To: /content/2018MT10770_B_model.zip
100% 402M/402M [00:03<00:00, 115MB/s]


In [ ]:
!unzip 2018MT10770_B_model.zip

Archive:  2018MT10770_B_model.zip
   creating: models/
  inflating: models/model-1635181112.7060943.pt  


In [ ]:
!pip install -q transformers

     |████████████████████████████████| 2.9 MB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 38.3 MB/s 
     |████████████████████████████████| 596 kB 44.0 MB/s 
     |████████████████████████████████| 895 kB 50.5 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 


# Imports

In [ ]:
## Import relevant packages

import os
import pandas as pd
import numpy as np

import torch

from torch.utils.data import Dataset, DataLoader
from torch.utils.data import RandomSampler

import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

from transformers import BertTokenizerFast
from transformers import BertForSequenceClassification
from transformers import get_scheduler, AdamW


from argparse import ArgumentParser
from pdb import set_trace
from tqdm import tqdm

import random
import json

In [ ]:
def seed_everything(seed=772):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
## Various utility functions

"""checking arguments"""
def check_args(args):
	# --tmp_dir
	check_folder(args.tmp_dir)

	# --data_path
	assert os.path.isfile(args.test_data_path), "test_data_path not found"

	# --saved_model
	assert os.path.isfile(args.saved_model), "saved_model not found"

	return args

def makedirs(name):
	"""helper function for python 2 and 3 to call os.makedirs()
		avoiding an error if the directory to be created already exists"""

	import os, errno

	try:
		os.makedirs(name)
	except OSError as ex:
		if ex.errno == errno.EEXIST and os.path.isdir(name):
			# ignore existing directory
			pass
		else:
			# a different error happened
			raise

def check_folder(log_dir):
	if not os.path.exists(log_dir):
		os.makedirs(log_dir)
	return log_dir


# Args

In [ ]:
def parse_args():
  parser = ArgumentParser(description='Word Meaning Comparison')

  parser.add_argument('--test_data_path', '-d', type=str, default='test.data.txt')
  parser.add_argument('--saved_model', '-m', type=str, default='/content/models/model-1635181112.7060943.pt')
  parser.add_argument('--out_path', '-o', type=str, default='output.txt')

  parser.add_argument('--tmp_dir', type=str, default='tmp')

  parser.add_argument('--gpu', type=int, default=0)

  return check_args(parser.parse_known_args()[0])

args = parse_args()

In [ ]:
device = torch.device('cuda' if args.gpu and torch.cuda.is_available() else 'cpu')

# Const

In [ ]:
label_encode = {
    'F': 0,
    'T': 1
}

label_decode = {
    0: 'F',
    1: 'T'
}

# Model 
(Class Definition)

In [ ]:
"""[not required]

class BertWSD(nn.Module)
  pass

"""

'[not required]\n\nclass BertWSD(nn.Module)\n  pass\n\n'

# Load saved model

In [ ]:
## YOUR testing code


save_model = torch.load(args.saved_model, map_location=device)


model = save_model['model']
model.load_state_dict(save_model['model_state'])
model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
save_model['args']

Namespace(add_train_data_path='my_wic_mcl_data/training.en-en.data', add_train_gold_path='my_wic_mcl_data/training.en-en.gold', add_val_data_path='my_wic_mcl_data/dev.en-en.data', add_val_gold_path='my_wic_mcl_data/dev.en-en.gold', batch_size=16, epochs=3, gpu=1, lr=2e-05, model='bert-base-cased', models_dir='models', out_dir='out', tmp_dir='tmp', train_data_path='data/train/train.data.txt', train_gold_path='data/train/train.gold.txt', val_data_path='data/validation/validation.data.txt', val_gold_path='data/validation/validation.gold.txt', vocab='bert-base-cased')

# Dataset

In [ ]:
class BertDataset(Dataset):
  def __init__(self, vocab, data_path, gold_path=None, json_format=False):
    self.label_encode = {
                          'F': 0,
                          'T': 1
                        }
    if vocab in ["bert-base-uncased", "bert-base-cased"]:
      self.tokenizer = BertTokenizerFast.from_pretrained(vocab)
    else:
      raise RuntimeError(f"Invalid voacb. Got: {vocab}")

    if json_format:    # for WiC-MLM dataset
      with open(data_path, 'r') as f:
        data = json.load(f)

      self.sentences = [(s['sentence1'], s['sentence2']) for s in data]
      self.idx_ranges = [((int(s['start1']), int(s['end1'])), (int(s['start2']), int(s['end2']))) for s in data]
      
      self.labels=None
      if gold_path is not None:
        with open(gold_path, 'r') as f:
          data = json.load(f)
        self.labels = [self.label_encode[s['tag']] for s in data]
        
    else:
      df = pd.read_csv(data_path, sep='\t', names=['word', 'pos', 'idx', 's1', 's2'])

      self.sentences = [(s1, s2) for s1, s2 in zip(df["s1"], df["s2"])]
      #self.idx_ranges = [((int(s1), int(e1)), (int(s2), int(e2))) for s1, e1, s2, e2 in zip(df["idx1_start"], df["idx1_end"], df["idx2_start"], df["idx2_end"])]
      
      self.labels=None
      if gold_path is not None:
        df = pd.read_csv(gold_path, names=['label'])
        df['label'] = df['label'].replace(self.label_encode)
        self.labels = df["label"].tolist()

    print(f"vocab: {vocab}, json: {json_format}")
    print(self.sentences[0],self.sentences[1],self.sentences[2])
    print(len(self.sentences))
                   
  def tokenize(self, s1, s2):
    return self.tokenizer(s1, s2,
                          add_special_tokens = True,
                          padding="max_length",
                          max_length=128,
                          #padding="longest",
                          truncation=True,
                          #return_offsets_mapping=True,
                          #return_tensors="pt"
                          )


  def __len__(self):
    return len(self.sentences)
  
  def __getitem__(self, index):
    s1, s2 = self.sentences[index]
    tokenized = self.tokenize(s1, s2)

    input_ids = torch.tensor(tokenized['input_ids'])
    token_type_ids = torch.tensor(tokenized['token_type_ids'])
    attention_mask = torch.tensor(tokenized['attention_mask'])

    label = self.labels[index] if self.labels is not None else -1

    return (input_ids, token_type_ids, attention_mask), torch.tensor(label, dtype=torch.int64).to(device)


In [ ]:
seed_everything()


test_data = BertDataset(data_path=args.test_data_path,
                         vocab=save_model['args'].vocab,
                         json_format=False,
                        )

test_dloader = DataLoader(test_data, 
                           batch_size=len(test_data), #args.batch_size,
                           shuffle=False, # important!
                           #pin_memory=True,
                           #num_workers=0,
                           )

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab: bert-base-cased, json: False
('Room and board .', 'He nailed boards across the windows .') ('Circulate a rumor .', 'This letter is being circulated among the faculty .') ('Hook a fish .', 'He hooked a snake accidentally , and was so scared he dropped his rod into the water .')
638


In [ ]:
if True:
  dt = BertDataset(data_path=args.test_data_path,
                   vocab=save_model['args'].vocab,
                   json_format=False,)
  #print(dt.tokenizer.decode(dt.__getitem__(0)[0]['input_ids'][0]))
  print(dt.__getitem__(0)[-1])
  #del dt

vocab: bert-base-cased, json: False
('Room and board .', 'He nailed boards across the windows .') ('Circulate a rumor .', 'This letter is being circulated among the faculty .') ('Hook a fish .', 'He hooked a snake accidentally , and was so scared he dropped his rod into the water .')
638
tensor(-1)


In [ ]:
for t in test_dloader:
  print(t[0][0].size())
  for el in t[0][0]:
    print(dt.tokenizer.decode(el))
  break


torch.Size([638, 128])
[CLS] Room and board. [SEP] He nailed boards across the windows. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] Circulate a rumor. [SEP] This letter is being circulated among the faculty. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

# Inference

In [ ]:
model.eval()
with torch.no_grad():
  for b in test_dloader:
      (input_ids, token_type_ids, attention_mask), labels = b
      input_ids, token_type_ids, attention_mask = input_ids.to(device), token_type_ids.to(device), attention_mask.to(device)
      pred = model(input_ids=input_ids,
                      token_type_ids=token_type_ids,
                      attention_mask=attention_mask,
                      ).logits
      pred = torch.argmax(pred, dim=1)
      input_ids, token_type_ids, attention_mask = input_ids.cpu(), token_type_ids.cpu(), attention_mask.cpu()


## Save output

In [ ]:
with open(args.out_path, 'w') as f:
  for i in pred:
    #print(label_decode[i.item()],file=f)
    print(label_decode.get(i.item(),'F'), file=f)

In [ ]:
## DONT CHANGE THIS CELL
# Your testing code must produce a file output.txt with predictions as T and F in each line

## Final Evaluation 
# this is currently the same as dev.gold.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
correct, total = 0., 0.
for prediction, gold in zip(open('output.txt'), open('test.gold.txt')):
  prediction, gold = prediction.strip(), gold.strip()
  total += 1
  if prediction == gold:
    correct += 1

## Report this as the final validation performance 
print('Performance = ', (correct/total))

--2021-10-25 17:06:31--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt [following]
--2021-10-25 17:06:32--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1276 (1.2K) [text/plain]
Saving to: ‘test.gold.txt’

test.gold.txt       100%[===================>]   1.25K  --.-KB/s    in 0s      

2021-10-25 17:06:33 (81.2 MB/s) - ‘test.gold.txt’ saved [1276/1276]

Performance =  0.7210031347962382
